# Finding similar items

## 2. Shingling of documents

Reference: 
* Information Retrieval Book https://nlp.stanford.edu/IR-book/html/htmledition/near-duplicates-and-shingling-1.html
* Elasticsearch Blog: https://www.elastic.co/blog/searching-with-shingles

Similar concepts: **n-gram** vs **shingles**

**n-gram** is a contiguous sequence of n items from a given sequence of text or speech. The items can be phonemes, syllables, letters, words or base pairs according to the application. When the items are words, n-grams may also be called **shingles**. [Wikipedia]

### 2.2 Choosing Shingling size

* K should be large enough that the probability of any given shingle appearing in any given document is low

### 2.3 Hashing Shingles

* Why hashing is needed? For data storage and query efficiency. 

In details, assuming we keep 9-shingles (9-char-length substring). For example, a shingle, "ABCDEFGHI", there are $27^{9}=3^{27}$ possible combinations. $2^{32} (~4mil)$

### 2.4 Singles built from words

* Stop words can be filtered out, or
* Combined with following words to give 3-word singles

## 3. Similarity-Preserving Summaries of Sets

* An observation: Storing shingles (n-gram is a more familiar concept) would cost memory n times the document itself. 
* Why? Because words are repeated in consecutive n-grams. E.g. the word "**are**" is repeated 3 times in consecutive 3-word shingles in sentence "hello how **are** you today": ["hello how **are**", "how **are** you", "**are** you today"]

    => It is impossibly large to store all of the shingles
    
        => Find a more compact set of "signatures"
        
### 3.1 Matrix representation of the sets
Concepts:
* Universal set (like vocabulary set of all documents) {a, b, c, d, e}
* Sets of interest: $S_1$={a, d}, and likewise for $S_2$, $S_3$, $S_4$. For example, they are 4 documents. 

Another way to think of this matrix is: rows representing items and columns representing customers.

### 3.2 Minhashing
* Signatures from large number of calculation, each is a minhash of the matrix above. 
    * From signatures, we can compare and estimate similarity (Jaccard, in this case) between sets.
* Minhash steps in principle: (1) pick a permutation of rows => (2) hash value = row number of first 1

For example, from the orginal order: (a, b, c, d, e) re-arrange the order of the rows to (b, e, a, c, d).

| Elements | $S_1$ | $S_2$|$S_3$|$S_4$
| :-:      |:-:    |:-:   |:-:  | :-:
| b | 0 | 0 | 1 | 0  
| e | 0 | 0 | 1 | 0
| a | 1 | 0 | 0 | 1 
| d | 1 | 0 | 1 | 1
| c | 0 | 1 | 0 | 1
The hash function apply in each set would give: $h(S_1)=a, h(S_2)=c, h(S_3)=b, h(S_4)=d$. 

### 3.3 Minhasing and Jaccard Similarity
By probability reasoning we have

$\text{Prob}(h(S_1)==h(S_2))=\frac{x}{x+y}$ where 
* $x$ is the number of rows where both columns have 1, 
* $y$ is the number of column where 1 appears in only one of the column of the pair.

### 3.4 Minhash Signatures
* Take a set S
* Take i.e. 100 hash function (permutation of the row index) and apply on the set S
* Get [$h_1(S), h_2(S), ..., h_n(S)$]
* Order them into **signature matrix**

## 3.5 Computing Minhash Signatures
* Coming to reality, not feasible to compute permutation of millions of rows. 
* Simulate by a hash function: n rows -> n buckets.  Accept collision and empty buckets. 
* Denote $SIG(i,c)$ is result of i-th hash function with set $S_c$
    * Consider row r of each column. 
    * Compute $h_1(r), h_2(r),...,h_n(r)$. For example: $h_1=3x+1 \text{ mod }5$
    * For each column c:
    
    ```
    if c[r] == 0: #(value of c in row r)
        pass
    elif c[r] == 1: 
        for i in range(1,n):
            SIG(i,c) = min(SIG(i,c), h_i(r))   Reducing the row to have the chance picking the min row
    ```

* Finally, obtain the signature matrix. Then apply Jaccard similarity function on the columns. 
    
    
|       | $S_1$ | $S_2$|$S_3$|$S_4$
| :-:   |:-:    |:-:   |:-:  | :-:
| $h_1$ | 1     | 3    | 0   | 1
| $h_2$ | 0     | 2    | 0   | 0